In [3]:
import sys
import json
import argparse
import random
import csv
import numpy as np
from scipy.signal import argrelextrema
from pandas import read_csv
import random
import os 
os.environ


import tensorflow as tf
import keras
import PIL
import numpy as np
import pandas as pd
import os
import math
from sklearn.model_selection import train_test_split
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Dropout, BatchNormalization
def read_file(file_path, type):
    if type == "csv":
        data = read_csv(file_path)
        return data.values
    elif type == "json":
        f = open(file_path)
        data = json.load(f)
        return data

In [4]:

argument_parser = argparse.ArgumentParser(description="A script used to replay the game session.")

# participants-path = .../toadstool/participants
# groundtruth-path = .../GroundTruth
argument_parser.add_argument("-p", "--participants-path", type=str, default=None)
argument_parser.add_argument("-i", "--participant-idx", type=str, default=None)
argument_parser.add_argument("-g", "--groundtruth-path", type=str, default=None)
argument_parser.add_argument("-o", "--output-path", type=str, default= None)


def read_file(file_path, type):
    if type == "csv":
        data = read_csv(file_path)
        return data.values
    elif type == "json":
        f = open(file_path)
        data = json.load(f)
        return data


if __name__ == "__main__":
    # parser
    args = argument_parser.parse_args()
    participants_path = args.participants_path
    participant_idx = args.participant_idx
    groundtruth_path = args.groundtruth_path
    output_path = f'{args.output_path}
    

    
    random_para = 10
    # file path
    session_path = f'{participants_path}/participant_{participant_idx}/participant_{participant_idx}_session.json'
    truth_path = f'{groundtruth_path}/truth/participant_{participant_idx}_events.json'
    yourPath = 'i5'

usage: ipykernel_launcher.py [-h] [-p PARTICIPANTS_PATH] [-i PARTICIPANT_IDX]
                             [-g GROUNDTRUTH_PATH] [-o OUTPUT_PATH]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/huangyushu/Library/Jupyter/runtime/kernel-cb2a0a28-1fae-4fba-9bb3-74b71b1b39b6.json


SystemExit: 2

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
   # event we have
    event = {0:"new_stage", 1:"flag_reached", 2:"status_up", 3:"status_down", 4:"life_lost" ,5:"nan"}
    out = {"new_stage":0, "flag_reached":1, "status_up":2, "status_down":3, "life_lost":4 ,"nan":5}
 # game frame number
    game = read_file(session_path, "json")#frame的每個數字
    game_len = int(game["obs_n"])

NameError: name 'read_file' is not defined

In [139]:
 game_len

125493

In [140]:
import os


allFileList = os.listdir(yourPath)
X_train = []
Y_train = []
P=[]
num=[]
for i in range(1,game_len+1):
    num.append(i)
    

for i in range(1,game_len+1):
        file='game_'+str(i)+'.png'
        p = yourPath + "/" +file
    #             img = cv2.imread(p)
        P.append(p) 

df = pd.DataFrame(P)
df['frame']= num
df

,0,frame
0,scripts/i5/game_1.png,1
1,scripts/i5/game_2.png,2
2,scripts/i5/game_3.png,3
3,scripts/i5/game_4.png,4
4,scripts/i5/game_5.png,5
...,...,...
125488,scripts/i5/game_125489.png,125489
125489,scripts/i5/game_125490.png,125490
125490,scripts/i5/game_125491.png,125491
125491,scripts/i5/game_125492.png,125492


In [141]:
model = keras.models.load_model('mariobalanced.h5')

In [142]:
d1 = df.loc[:10000]
d2 = df.loc[10001:20000]
d3 = df.loc[20001:30000]
d4 = df.loc[30001:40000]
d5 = df.loc[40001:50000]
d6 = df.loc[50001:60000]
d7 = df.loc[60001:70000]
d8 = df.loc[70001:80000]
d9 = df.loc[80001:90000]
d10 = df.loc[90001:100000]
d11 = df.loc[100001:110000]
d12 = df.loc[110001:120000]
d13 = df.loc[120001:(game_len-1)]
datalist = [d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13]

In [143]:

def predict(data):
    X = data[0]
    image_list = []
    for file_name in X:
        image_array = np.array(PIL.Image.open(file_name))
        resized_image_array = cv2.resize(image_array, (128, 128))
        image_list.append(resized_image_array)
    train_images = np.array(image_list)

    prediction = model.predict(train_images)
    result = []

    for i in prediction:
            result.append(np.argmax(i))

    re=[]
    for i in result:
        re.append(event[i])
    data['result']=re
    filt = (data['result'] != "nan")
    data = data.loc[filt]
    final = pd.concat([data['frame'], data['result']], axis = 1)
    
    return final


In [144]:
final = []
for i in range(13):
    f = predict(datalist[i])
    final.append(f)
    print(i+1,"fin")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1 fin
2 fin
3 fin
4 fin
5 fin
6 fin
7 fin
8 fin
9 fin
10 fin
11 fin
12 fin
13 fin


In [145]:
output = pd.concat([final[0],final[1],final[2],final[3],final[4],final[5],final[6],final[7],final[8],final[9],final[10],final[11],final[12]], axis = 0)
output

,frame,result
937,938,status_up
1953,1954,status_up
1954,1955,status_up
1955,1956,status_up
2904,2905,status_down
...,...,...
123769,123770,life_lost
123770,123771,life_lost
123771,123772,life_lost
125335,125336,status_up


In [146]:
    # write output
    output_path='run/participant_5_events.csv'
    output.to_csv(output_path, index = False,header=False)